<a href="https://colab.research.google.com/github/JoYongJIn/YongJin-Repository/blob/main/%08Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
This code has been brought from the kaggle below and has been slightly modified.:

https://www.kaggle.com/code/yeonmin/solution
'''

In [ ]:
'''
dataset의 sample수는 262,144개이고 모델의 성능을 평가하는 test data는 131,073개입니다.
feature(속성)수가 258개인 데이터를 0또는 1 두 범주 중 하나로 분류하는 것이 목적입니다.
'''

In [ ]:
# Stacking을 위해 필요한 모듈과 라이브러리를 import합니다.
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold

from sklearn.decomposition import KernelPCA
from sklearn.mixture import GaussianMixture as GMM
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import lightgbm as lgbm

In [ ]:
# 히스토그램 기반 모델로 데이터에 대한 분석 및 예측을 수행하는 클레스를 만듭니다.
#(주어진 데이터의 각 변수에 대해 히스토그램을 만들고, 새로운 데이터의 각 관측치가 해당 히스토그램의 어느 bin에 속하는지를 판단하여 점수를 계산하는 방식으로 작동합니다.)
class hist_model(object): # 설계도 역할을 합니다.

    def __init__(self, bins=50): # 클래스의 생성자init으로 새로운 'hist_model'객체를 만듭니다.
        self.bins = bins # bins라는 매개변수를 가지며, 기본값은 50입니다.

    def fit(self, X): # fit 메소드는 모델을 데이터에 맞추기 위해 사용됩니다. 'X'는 훈련 데이터셋입니다.

        bin_hight, bin_edge = [], [] # 히스토그램의 높이와 경계(interval)를 저장할 빈리스트를 생성합니다.

        for var in X.T: # 'X'의 각 열에대해 반복합니다. 이때.T는 'X'의 전치(transpose)로 행과 열을 바꿉니다.
            # get bins hight and interval
            bh, bedge = np.histogram(var, bins=self.bins) # self bin의 수를 이용하여 var의 히스토그램을 계산합니다.
            bin_hight.append(bh)
            bin_edge.append(bedge) # 계산된 히스토그램 높이와 경계를 각각 리스트에 추가합니다.

        self.bin_hight = np.array(bin_hight)
        self.bin_edge = np.array(bin_edge) # 리스트를 Numpy 배열로 변환하여 클래스 속성으로 저장합니다.

    def predict(self, X): # predict 메소드는 새로운 데이터'X'에 대해 예측을 수행합니다.

        scores = [] # 예측 점수를 저장할 빈 리스트를 생성합니다.
        for obs in X: # 'X'의 각 관측치 obs(observation)에 대해 반복합니다.
            obs_score = [] # 개별 관측치의 점수를 저장할 리스트를 생성합니다.
            for i, var in enumerate(obs): # 관측치의 각 변수 var에대해 반복합니다. enumerate는 객체를 순회하며 각 요소의 인덱스와 값을 동시에 반환하도록 해줍니다.(반복 작업중 요소의 순서나 위치를 추적할때 유용합니다.)
                # find wich bin obs is in
                bin_num = (var > self.bin_edge[i]).argmin()-1 # 해당 변수 값이 어느 히스토그램에 속하는지 찾습니다.
                obs_score.append(self.bin_hight[i, bin_num]) # 찾은 bin의 높이에 해당 관측치의 점수로 사용하여 'obs_score'리스트에 추가합니다.

            scores.append(np.mean(obs_score)) # 각 관측치의 평균점수를 scores리스트에 추가합니다.

        return np.array(scores) # 완료된 scores리스트를 Numpy배열로 변환하여 반환합니다.


In [ ]:
# 모델함수를 정의합니다.
def run_model(clf_list, train, test, random_state, gmm_init_params='kmeans'): # 변수들을 입력합니다.
    # 입력 파라미터에는 분류기목록, 훈련데이터셋, 테스트데이터셋, 무작위상태, 가우시안혼합모델의 초기파라미터가 있습니다.
    MODEL_COUNT = len(clf_list) # clf_list의 길이를 계산하여 모델의 수를 저장합니다.

    oof_train = np.zeros((len(train), MODEL_COUNT)) # 훈련데이터셋에대핸 OOF(out of fold) 예측 결과를 저장할 배열을 초기화해둡니다.
    oof_test = np.zeros((len(test), MODEL_COUNT)) # 테스트데이터셋에대핸 OOF(out of fold) 예측 결과를 저장할 배열을 초기화해둡니다.
    train_columns = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]
    # 훈련데이터셋에서 'id', 'target', 'wheezy-copper-turtle-magic'을 제외한 모든 열의 이름을 추출합니다.
    for magic in tqdm_notebook(range(512)): # 0부터 511까지의 숫자에 대해 랜덤으로 반복하며, tqdm_notebook을 사용하여 진행률을 시각적으로 표현합니다.
        x_train = train[train['wheezy-copper-turtle-magic'] == magic] # w-c-t-m열이 magic과 일치하는 행을 훈련 및 테스트 데이터셋으로 선택합니다.
        x_test = test[test['wheezy-copper-turtle-magic'] == magic]
        print("Magic: ", magic, x_train.shape, x_test.shape) # magic과 행의수를 출력하여 확인합니다.

        train_idx_origin = x_train.index
        test_idx_origin = x_test.index # 선택된 훈련, 테스트 데이터셋에 원래 인덱스를 저장합니다.

        train_std = x_train[train_columns].std()
        cols = list(train_std.index.values[np.where(train_std >2)]) # 훈련 데이터의 표준편차를 계산하고, 표준편차가 2보다 큰 열을 선택합니다.

        x_train = x_train.reset_index(drop=True)
        y_train = x_train.target # 훈련데이터의 인덱스를 초기화 하고, 타겟변수를 분리합니다.

        x_train = x_train[cols].values
        x_test = x_test[cols].values # 선택된 열만을 가지고 훈련 및 테스트 데이터를 재구성합니다.

        all_data = np.vstack([x_train, x_test]) # 훈련 및 테스트 데이터를 수직으로쌓아 하나의 데이텃셋으로 만듭니다.
        # print("all_data: ", all_data.shape)
        # Kernel PCA
        all_data = KernelPCA(n_components=len(cols), kernel='cosine', random_state=random_state).fit_transform(all_data)
        # 커널 PCA(공분산행렬이용)를 사용하여 데이터의 차원을 축소합니다.
        # GMM
        gmm = GMM(n_components=5, random_state=random_state, max_iter=1000, init_params=gmm_init_params).fit(all_data)
        gmm_pred = gmm.predict_proba(all_data)
        gmm_score = gmm.score_samples(all_data).reshape(-1, 1)
        gmm_label = gmm.predict(all_data) # GMM모델(모수이용 군집화)을 학습시키고 데이터에 대한 확률, 점수, 라벨을 계산합니다.

        # hist feature
        hist = hist_model()
        hist.fit(all_data)
        hist_pred = hist.predict(all_data).reshape(-1, 1) # hist_model객체를 생성하고 all_data에대해 학습을 하고 예측을 수행합니다.

        all_data = np.hstack([all_data, gmm_pred, gmm_pred, gmm_pred, gmm_pred, gmm_pred])
        # GMM 모델의 예측 결과를 'all_data'에 여러번 추가합니다. (stacking입니다.)
        # Add Some Features
        all_data = np.hstack([all_data, hist_pred, gmm_score, gmm_score, gmm_score])
        # 히스토그램 예측 결과와 GMM모델의 점수를 'all_data'에 추가합니다.
        # STANDARD SCALER
        all_data = StandardScaler().fit_transform(all_data) # 데이터를 표준화 합니다.(평균0, 표준편차1)

        # new train/test
        x_train = all_data[:x_train.shape[0]]
        x_test = all_data[x_train.shape[0]:] # 표준화된 데이터를 다시 훈련 및 테스트 데이터셋으로 분할합니다.
        # print("data size: ", x_train.shape, x_test.shape)
        fold = StratifiedKFold(n_splits=5, random_state=random_state) # 5겹 교차검증을 위한 객체(StratifiedKFold)를 생성합니다.
        for trn_idx, val_idx in fold.split(x_train, gmm_label[:x_train.shape[0]]):
            for model_index, clf in enumerate(clf_list):
                clf.fit(x_train[trn_idx], y_train[trn_idx])
                oof_train[train_idx_origin[val_idx], model_index] = clf.predict_proba(x_train[val_idx])[:,1]
                # 모델을 사용하여 검증 데이터의 클래스 1에 대한 예측 확률을 계산하고, 이를 'oof_train'에 저장합니다.
                # 2023/03/02 데이터의 형식이 변경되어, x_test 예측 시 오류 발생하는 것 수정
                if x_test.shape[0] == 0:
                    continue

                #print(oof_test[test_idx_origin, model_index].shape)
                #print(x_test.shape)
                #print(clf.predict_proba(x_test)[:,1])
                oof_test[test_idx_origin, model_index] += clf.predict_proba(x_test)[:,1] / fold.n_splits
                # 테스트 데이터에 대한 예측을 수행하고 평균 예측확률을 off_test에 저장합니다.
    for i, clf in enumerate(clf_list): # 모든 분류기에 대해 반복합니다.
        print(clf) # 현재 분류기를 출력합니다.
        print(roc_auc_score(train['target'], oof_train[:, i])) # 훈련 데이터에 대한 모델의 성능을 ROC AUC점수로 계산하고 출력합니다.
        print()

    oof_train_df = pd.DataFrame(oof_train)
    oof_test_df = pd.DataFrame(oof_test) # 두 결과를 Pandas DataFrame으로 변환합니다.

    return oof_train_df, oof_test_df # 두 결과를 반환합니다.

In [ ]:
os.listdir('../input/instant-gratification/') # os모듈을 사용하여 특정 디렉토리 내의 파일과 디렉토리 목록을 list형태로 가져옵니다.

In [ ]:
train = pd.read_csv('../input/instant-gratification/train.csv')
test = pd.read_csv('../input/instant-gratification/test.csv') # csv파일로 읽어 데이터프레임으로 변환합니다.

In [ ]:
svnu_params = {'probability':True, 'kernel':'poly','degree':4,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}
# Support Vector Machine with Nu모델의 파라미터를 설정합니다. 확률추정활성화, 다항식커널사용, 커널의 차수4, 커널계수를 auto로하여 자동으로 결정, nu파라미터 0.4, 커널함수의 독립항 0.08, 난수 생성기 시드4
svnu2_params = {'probability':True, 'kernel':'poly','degree':2,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}
qda_params = {'reg_param':0.111} # QDA모델의 파리미터를 설정합니다. (정규화 파라미터를 0.111로 설정)
svc_params = {'probability':True,'kernel':'poly','degree':4,'gamma':'auto', 'random_state':4}
neighbor_params = {'n_neighbors':16} # 이웃의 수를 16으로합니다.
lr_params = {'solver':'liblinear','penalty':'l1','C':0.05,'random_state':42} # L1정규화를 사용하고 정규화 강도를 0.05로 합니다.


In [ ]:
nusvc_model = svm.NuSVC(**svnu_params)
nusvc2_model = svm.NuSVC(**svnu2_params) # 모델생성
qda_model = QuadraticDiscriminantAnalysis(**qda_params)
svc_model = svm.SVC(**svc_params)
knn_model = neighbors.KNeighborsClassifier(**neighbor_params)
lr_model = linear_model.LogisticRegression(**lr_params)

model_list = [nusvc_model, nusvc2_model, qda_model, svc_model, knn_model, lr_model] # 모델리스트 생성
oof_train_kmeans_seed1, oof_test_kmeans_seed1 = run_model(model_list, train, test, 1)
oof_train_kmeans_seed2, oof_test_kmeans_seed2 = run_model(model_list, train, test, 2)
oof_train_random_seed1, oof_test_random_seed1 = run_model(model_list, train, test, 1, 'random')
oof_train_random_seed2, oof_test_random_seed2 = run_model(model_list, train, test, 2, 'random')

In [ ]:
train_second = (oof_train_kmeans_seed1 + oof_train_kmeans_seed2 + oof_train_random_seed1 + oof_train_random_seed2)/4
test_second = (oof_test_kmeans_seed1 + oof_test_kmeans_seed2 + oof_test_random_seed1 + oof_test_random_seed2)/4
print('Ensemble', roc_auc_score(train['target'], train_second.mean(1)))
# 앞서 생성된 모델들의 예측 결과를 평균내어 앙상블을 형성하고, 그 성능을 평가합니다.

In [ ]:
lgbm_meta_param = {
        #'bagging_freq': 5,
        #'bagging_fraction': 0.8,
        'min_child_weight':6.790,
        "subsample_for_bin":50000,
        'bagging_seed': 0,
        'boost_from_average':'true',
        'boost': 'gbdt',
        'feature_fraction': 0.450,
        'bagging_fraction': 0.343,
        'learning_rate': 0.025,
        'max_depth': 10,
        'metric':'auc',
        'min_data_in_leaf': 78,
        'min_sum_hessian_in_leaf': 8,
        'num_leaves': 18,
        'num_threads': 8,
        'tree_learner': 'serial',
        'objective': 'binary',
        'verbosity': 1,
        'lambda_l1': 7.961,
        'lambda_l2': 7.781
        #'reg_lambda': 0.3, # lgbm의 파라미터(학습방법과속도, 트리구조, 오버피팅방지관련, 정규화 등)들을 정의합니다.
    }

mlp16_params = {'activation':'relu','solver':'lbfgs','tol':1e-06, 'hidden_layer_sizes':(16, ), 'random_state':42}
# 다층 퍼셉트론(Multi Layer Perceptron)을 구성하기 위한 파라미터를 설정합니다.

In [ ]:
SEED_NUMBER = 4
NFOLD = 5 # seed_num과 nfold를 설정합니다.

y_train = train['target']
oof_lgbm_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_lgbm_meta_test = np.zeros((len(test), SEED_NUMBER))
oof_mlp_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_mlp_meta_test = np.zeros((len(test), SEED_NUMBER)) # 훈련데이터셋과 테스트셋에 대한 메타 레벨 예측을 저장할 배열을 초기화 합니다.

for seed in range(SEED_NUMBER):
    print("SEED Ensemble:", seed)
    mlp16_params['random_state'] = seed
    lgbm_meta_param['seed'] = seed
    folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=seed) # nfold만큼의 부분으로 나눕니다. 모델의 일반화 능력 테스트를 도와줍니다.
    for fold_index, (trn_index, val_index) in enumerate(folds.split(train_second, y_train), 1):
        print(f"{fold_index} FOLD Start")
        trn_x, trn_y = train_second.iloc[trn_index], y_train.iloc[trn_index]
        val_x, val_y = train_second.iloc[val_index], y_train.iloc[val_index] # 검증데이터에 대한 예측을 수행합니다.

        mlp_meta_model = neural_network.MLPClassifier(**mlp16_params) # mlp모델 훈련 및 예측입니다.
        mlp_meta_model.fit(trn_x, trn_y)

        oof_mlp_meta_train[val_index, seed] = mlp_meta_model.predict_proba(val_x)[:,1]
        oof_mlp_meta_test[:, seed] += mlp_meta_model.predict_proba(test_second)[:,1]/NFOLD
        print("MLP META SCORE: ", roc_auc_score(val_y, oof_mlp_meta_train[val_index, seed])) # 성능을 봅니다.

        # lgbm meta model
        dtrain = lgbm.Dataset(trn_x, label=trn_y, silent=True)
        dcross = lgbm.Dataset(val_x, label=val_y, silent=True) # lightGBM모델을 훈련및 예측합니다.

        lgbm_meta_model = lgbm.train(lgbm_meta_param, train_set=dtrain, valid_sets=[dtrain, dcross],
                                     verbose_eval=False, early_stopping_rounds=100)

        oof_lgbm_meta_train[val_index, seed] = lgbm_meta_model.predict(val_x)
        oof_lgbm_meta_test[:, seed] += lgbm_meta_model.predict(test_second)/NFOLD
        print("LGBM META SCORE: ", roc_auc_score(val_y, oof_lgbm_meta_train[val_index, seed])) # ROC-AUC 점수로 평가합니다.

In [ ]:
oof_lgbm_meta_train_df = pd.DataFrame(oof_lgbm_meta_train).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_lgbm_meta_test_df = pd.DataFrame(oof_lgbm_meta_test).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_mlp_meta_train_df = pd.DataFrame(oof_mlp_meta_train).mean(axis=1).to_frame().rename(columns={0:'mlp'})
oof_mlp_meta_test_df = pd.DataFrame(oof_mlp_meta_test).mean(axis=1).to_frame().rename(columns={0:'mlp'})
# 앞에서한 LightBGM모델과 MLP모델의 OFF 예측 결과를 데이터프레임으로 변환하고 각 모델에 대한 평균 예측값을 계산하여 데이터프레임으로 저장합니다.

In [ ]:
oof_train_third = pd.concat([train_second, oof_lgbm_meta_train_df, oof_mlp_meta_train_df], axis=1)
oof_test_third = pd.concat([test_second, oof_lgbm_meta_test_df, oof_mlp_meta_test_df], axis=1)
# train_second, oof_lgbm_meta_train_df, oof_mlp_meta_train_df 예측 결과를 수평적으로(열방향으로 같은행들을 더함) 결합합니다다
print('Ensemble', roc_auc_score(train['target'], oof_train_third.mean(1)))
# 앞에서 생성된 여러 앙상블 모델의 예측결과를 결합하고, 이를 사용하여 최종 앙상블 모델의 성능을 평가한니다.

In [ ]:
submission = pd.read_csv('../input/instant-gratification/sample_submission.csv') # 대회에서 제공하는 샘플 제출양식을 포함한 csv파일을 읽어옵니다.
submission["target"] = oof_test_third.mean(1) # off_test_third.mean의 행별 평균을 계산하고 이를 submission데이터 프레임의 target열에 저장합니다.
submission.to_csv("submission.csv", index=False) # 데이터 프레임을 저장하여 제출하면됩니다.
# 모델의 최종 예측결과를 파일형식(대회양식)에 맞게 생성하여 제출합니다.